<a href="https://colab.research.google.com/github/VeronikaShe/ML-study_Recommendation/blob/main/hw_%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%86%D0%B8%D0%B8_%D0%BD%D0%B0_%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D0%B5_%D1%81%D0%BE%D0%B4%D0%B5%D1%80%D0%B6%D0%B0%D0%BD%D0%B8%D1%8F_shemyakina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy==1.26.4

In [2]:
!pip install scikit-surprise

In [3]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [4]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from tqdm import tqdm_notebook
import itertools

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

In [5]:
raw_data_movies = pd.read_csv('https://raw.githubusercontent.com/VeronikaShe/ML-study_Recommendation/refs/heads/main/movies.csv')
raw_data_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
raw_data_ratings = pd.read_csv('https://raw.githubusercontent.com/VeronikaShe/ML-study_Recommendation/refs/heads/main/ratings.csv')
raw_data_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
raw_data_tags = pd.read_csv('https://raw.githubusercontent.com/VeronikaShe/ML-study_Recommendation/refs/heads/main/tags.csv')
raw_data_tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [8]:
raw_data_links = pd.read_csv('https://raw.githubusercontent.com/VeronikaShe/ML-study_Recommendation/refs/heads/main/links.csv')
raw_data_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [9]:
joined_ratings = raw_data_ratings.join(raw_data_movies.set_index('movieId'), on='movieId')

In [10]:
joined_ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [11]:
title_mean_rating = {}

for title, group in tqdm_notebook(joined_ratings.groupby('title')):
    title_mean_rating[title] = group.rating.mean()

/tmp/ipython-input-174582896.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for title, group in tqdm_notebook(joined_ratings.groupby('title')):


  0%|          | 0/9719 [00:00<?, ?it/s]

In [12]:
title_mean_rating ['Toy Story (1995)']

3.9209302325581397

In [13]:
title_median_rating = {}

for title, group in tqdm_notebook(joined_ratings.groupby('title')):
    title_median_rating[title] = group.rating.median()

/tmp/ipython-input-985702825.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for title, group in tqdm_notebook(joined_ratings.groupby('title')):


  0%|          | 0/9719 [00:00<?, ?it/s]

In [14]:
title_median_rating ['Toy Story (1995)']

4.0

In [15]:
user_mean_rating = {}

for user_id, group in tqdm_notebook(joined_ratings.groupby('userId')):
    user_mean_rating[user_id] = group.rating.mean()

/tmp/ipython-input-1303064441.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for user_id, group in tqdm_notebook(joined_ratings.groupby('userId')):


  0%|          | 0/610 [00:00<?, ?it/s]

In [16]:
user_mean_rating [1]

4.366379310344827

In [17]:
user_median_rating = {}

for user_id, group in tqdm_notebook(joined_ratings.groupby('userId')):
    user_median_rating[user_id] = group.rating.median()

/tmp/ipython-input-131834240.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for user_id, group in tqdm_notebook(joined_ratings.groupby('userId')):


  0%|          | 0/610 [00:00<?, ?it/s]

In [18]:
user_median_rating [1]

5.0

In [19]:
# выводим топ 20 фильмов по средним оценкам
list(sorted(title_mean_rating, key=lambda x: x[1], reverse=True))[:20]

['Güeros (2014)',
 'Déjà Vu (1997)',
 'Déjà Vu (Deja Vu) (2006)',
 'Léon: The Professional (a.k.a. The Professional) (Léon) (1994)',
 'Ménage (Tenue de soirée) (1986)',
 'Pépé le Moko (1937)',
 'Azumi (2003)',
 'Oz the Great and Powerful (2013)',
 'By the Gun (2014)',
 'By the Law (1926)',
 'Byzantium (2012)',
 'Cyborg (1989)',
 'Cyrano de Bergerac (1990)',
 'Cyrus (2010)',
 'Dying Young (1991)',
 'Dying of the Light (2014)',
 'Dylan Moran: Monster (2004)',
 'Eye See You (D-Tox) (2002)',
 'Eye for an Eye (1996)',
 'Eye for an Eye, An (1981)']

In [20]:
movies_with_ratings = raw_data_movies.join(raw_data_ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [21]:
movies_with_ratings_and_tags = pd.merge(movies_with_ratings, raw_data_tags, on=['movieId','userId'])
movies_with_ratings_and_tags.dropna(inplace=True)

In [23]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [24]:
movie_tags = [raw_data_tags.tag.values]
movie_genres_tags[:5]

[]

In [55]:
genre_tag_strings = []
movies = []

for movie, group in tqdm_notebook(movies_with_ratings_and_tags.groupby('title')):
    genre_tag_strings.append(' '.join([str(s).replace(' ', '').replace('-', '') for s in (group.tag + group.genres).values]))
    movies.append(movie)

/tmp/ipython-input-3315984237.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for movie, group in tqdm_notebook(movies_with_ratings_and_tags.groupby('title')):


  0%|          | 0/1464 [00:00<?, ?it/s]

In [57]:
genre_tag_strings[:5]

['artisticComedy|Drama|Romance FunnyComedy|Drama|Romance humorousComedy|Drama|Romance inspiringComedy|Drama|Romance intelligentComedy|Drama|Romance quirkyComedy|Drama|Romance romanceComedy|Drama|Romance ZooeyDeschanelComedy|Drama|Romance',
 'lawyersDrama|Thriller',
 'creepyThriller suspenseThriller',
 'ShakespearesortofComedy|Romance',
 'dogsAdventure|Children|Comedy remakeAdventure|Children|Comedy']

In [58]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(genre_tag_strings)

In [59]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [60]:
neigh = NearestNeighbors(n_neighbors=10, n_jobs=-1, metric='manhattan')
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='manhattan', n_jobs=-1, n_neighbors=10)

In [62]:
test = change_string('lawyersDrama|Thriller')

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = neigh.kneighbors(X_tfidf2, return_distance=True)

In [63]:
res

(array([[0.        , 0.53545824, 1.87530547, 1.87530547, 1.87530547,
         1.88316135, 1.88316135, 1.88316135, 1.88316135, 1.88316135]]),
 array([[   1, 1280, 1428, 1367,  968,  612,  793,  334,  301,  850]]))

In [65]:
for i in res[1][0]:
    print(movies[i])

...And Justice for All (1979)
The Devil's Advocate (1997)
With a Friend Like Harry... (Harry, un ami qui vous veut du bien) (2000)
Vanishing, The (1993)
Phone Booth (2002)
In the Line of Fire (1993)
Mean Creek (2004)
Dolores Claiborne (1995)
Day of the Jackal, The (1973)
Murder in the First (1995)


In [42]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings_and_tags.userId,
    'iid': movies_with_ratings_and_tags.title + movies_with_ratings_and_tags.tag,
    'rating': movies_with_ratings_and_tags.rating
})

In [43]:
dataset.head()

,uid,iid,rating
0,336.0,Toy Story (1995)pixar,4.0
1,474.0,Toy Story (1995)pixar,4.0
2,567.0,Toy Story (1995)fun,3.5
3,62.0,Jumanji (1995)fantasy,4.0
4,62.0,Jumanji (1995)magic board game,4.0


In [44]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [45]:
trainset, testset = train_test_split(data, test_size=.15)

In [46]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [47]:
test_pred = algo.test(testset)

In [48]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8216


0.8216317943542754

In [50]:
algo.predict(uid=2, iid='Toy Story (1995)pixar')

Prediction(uid=2, iid='Toy Story (1995)pixar', r_ui=None, est=5.0, details={'actual_k': 0, 'was_impossible': False})